# CIA Factbook: SQL analysis of countries and their statistics

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. 

The Factbook contains demographic information for each country such as: 

- `population`: Total population
- `population_growth`:  Annual population growth rate, as a percentage
- `area`: Total land and water area

This dataset is a SQLite Database named factbook.db and can be dowloaded by clicking [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db).

We will now connect this database file to our Juypter.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Initial Exploration

As per our SQL query above against the database, we can see that the table we are going to use for our data exploration is **facts**.

Let's now see some rows stored in that table to understand the structure a bit better.

In [3]:
%%sql
SELECT *
  FROM facts
  LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


From the above display of sample rows, and from information online through [CIA Factbook](https://www.cia.gov/the-world-factbook/references/definitions-and-notes/), the below description for some of the columns would help us in our analysis going forward.


- `name`: Name of the country
- `area`: Country's total area (both land and water)
- `area_land` Country's land area in square kilometers
- `area_water`: Country's waterarea in square kilometers
- `population`: Country's population
- `population_growth`: Country's population growth as a percentage
- `birth_rate`: Country's birth rate, or the number of births per year per 1,000 people
- `death_rate`: Country's death rate, or the number of death per year per 1,000 people.

## Data Analysis

Let's now understand the minimum and maximum values of two key demographic columns - `population` and `population_growth`.

In [4]:
%%sql
SELECT MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
  FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics:


- There's a country with a population of **0**
- There's a country with a population of **7256490011** - Or more than 7.2 billion people

Let's drill into that.

In [5]:
%%sql
SELECT *
  FROM facts
  WHERE population = (SELECT MIN(population) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [6]:
%%sql
SELECT *
  FROM facts
  WHERE population = (SELECT MAX(population) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. 

It also seems like the table contains a row for Antarctica, which explains the population of 0.

Now, let's exclude the row for the whole world and calculate the summary statistics again.

In [7]:
%%sql
SELECT MIN(population) 'Minimum Population',MAX(population) 'Maximum Population',MIN(population_growth) 'Minimum Population Growth',MAX(population_growth) 'Maximum Population Growth'
  FROM facts
  WHERE name != 'World';

Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,1367485388,0.0,4.02


## Answering curious questions from the data

### 1. What's the country with the maximum population?

In [8]:
%%sql
SELECT *
  FROM facts
  WHERE population = (SELECT MAX(population)
  FROM facts
  WHERE name != 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


### 2. What's the average population and average total area across all countries?

In [9]:
%%sql
SELECT ROUND(AVG(population),2) 'Average Population',ROUND(AVG(area),2) 'Average Total Area'
  FROM facts
  WHERE name != 'World';

Done.


Average Population,Average Total Area
32242666.57,555093.55


### 3. Which countries have population that is greater than average at the same time has area less than average?

In [10]:
%%sql
SELECT id, code, name, population, area
  FROM facts
  WHERE population > (SELECT AVG(population) FROM facts
                       WHERE name != 'World')
    AND area < (SELECT AVG(area) FROM facts
                       WHERE name != 'World')
  ORDER BY population DESC, area ASC;

Done.


id,code,name,population,area
14,bg,Bangladesh,168957745,148460
85,ja,Japan,126919659,377915
138,rp,Philippines,100998376,300000
192,vm,Vietnam,94348835,331210
65,gm,Germany,80854408,357022
173,th,Thailand,67976405,513120
185,uk,United Kingdom,64088222,243610
83,it,Italy,61855120,301340
91,ks,"Korea, South",49115196,99720
163,sp,Spain,48146134,505370


### 4. List top 5 countries that have more water than land?

In [11]:
%%sql
SELECT id, code, name, CAST(area_water AS Float) water_area, CAST(area_land AS Float) land_area, CAST(area AS Float) total_area, ROUND((CAST(area_water AS Float)/CAST(area AS Float))*100,2) water_area_ratio
  FROM facts
  WHERE area_water IS NOT NULL
    AND area IS NOT NULL
    AND area_water != 0
    AND area != 0
  ORDER BY water_area_ratio DESC
  LIMIT 5;

Done.


id,code,name,water_area,land_area,total_area,water_area_ratio
228,io,British Indian Ocean Territory,54340.0,60.0,54400.0,99.89
247,vq,Virgin Islands,1564.0,346.0,1910.0,81.88
246,rq,Puerto Rico,4921.0,8870.0,13791.0,35.68
12,bf,"Bahamas, The",3870.0,10010.0,13880.0,27.88
71,pu,Guinea-Bissau,8005.0,28120.0,36125.0,22.16


### 5. Which country has the most people and which country has the highest growth rate?

In [12]:
%%sql
SELECT *
  FROM facts
  WHERE population = (SELECT MAX(population) FROM facts WHERE name != 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


That is something we could have predicted. It's **China** with the most people at 1.36 billion. Now, let's see which country has the highest growth rate.

In [13]:
%%sql
SELECT *
  FROM facts
  WHERE population_growth = (SELECT MAX(population_growth) FROM facts WHERE name != 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


### 6. Which countries will add the most people to their populations next year?

In [14]:
%%sql
SELECT id, code, name, population, population_growth, 
       ROUND(CAST(population * (population_growth/100) AS Float),2) population_addition
  FROM facts
  WHERE name != 'World'
  ORDER BY population_addition DESC
  LIMIT 5;

Done.


id,code,name,population,population_growth,population_addition
77,in,India,1251695584,1.22,15270686.12
37,ch,China,1367485388,0.45,6153684.25
129,ni,Nigeria,181562056,2.45,4448270.37
132,pk,Pakistan,199085847,1.46,2906653.37
58,et,Ethiopia,99465819,2.89,2874562.17


As per the above, we see that India is going to add more people to it's population based on its current population and the population growth rate.

### 7. Which countries have a higher death rate than birth rate?

In [15]:
%%sql
SELECT id ID, name 'Country', birth_rate 'Birth Rate', death_rate 'Death Rate', ROUND(CAST(death_rate - birth_rate AS Float),2) 'Death Birth Rate Difference'
  FROM facts
  WHERE name != 'World'
  ORDER BY 5 DESC
  LIMIT 5;

Done.


ID,Country,Birth Rate,Death Rate,Death Birth Rate Difference
26,Bulgaria,8.92,14.44,5.52
153,Serbia,9.08,13.66,4.58
96,Latvia,10.0,14.31,4.31
102,Lithuania,10.1,14.27,4.17
183,Ukraine,10.72,14.46,3.74


This seems to point that the death rate is higher than the birth rate across countries for these - Strangely, all of these Eastern European countries.

### 8. Top 10 countries with highest population/area ratio or Density of population?

In [16]:
%%sql
SELECT id ID, name 'Country', population 'Population', area 'Total Area', CAST(population/area AS Float) 'Population per square kms'
  FROM facts
  WHERE name != 'World'
  ORDER BY 5 DESC
  LIMIT 10;

Done.


ID,Country,Population,Total Area,Population per square kms
205,Macau,592731,28,21168.0
117,Monaco,30535,2,15267.0
156,Singapore,5674472,697,8141.0
204,Hong Kong,7141106,1108,6445.0
251,Gaza Strip,1869055,360,5191.0
233,Gibraltar,29258,6,4876.0
13,Bahrain,1346613,760,1771.0
108,Maldives,393253,298,1319.0
110,Malta,413965,316,1310.0
227,Bermuda,70196,54,1299.0


This list is influenced by the population and total area irrespective of any minimum population figures.

We will now check the population density for most populous countries - say minimum population of atleast 8,000,000 people.

### 9. Top 10 most populous countries by density

In [17]:
%%sql
SELECT id ID, name 'Country', population 'Population', area 'Total Area', CAST(population/area AS Float) 'Population per square kms'
  FROM facts
  WHERE name != 'World'
    AND population > 8000000
  ORDER BY 5 DESC
  LIMIT 10;

Done.


ID,Country,Population,Total Area,Population per square kms
14,Bangladesh,168957745,148460,1138.0
196,Taiwan,23415126,35980,650.0
91,"Korea, South",49115196,99720,492.0
144,Rwanda,12661733,26338,480.0
125,Netherlands,16947904,41543,407.0
82,Israel,8049314,20770,387.0
29,Burundi,10742276,27830,385.0
77,India,1251695584,3287263,380.0
17,Belgium,11323973,30528,370.0
73,Haiti,10110019,27750,364.0


## Conclusion

In this data analysis through SQL:

We reaffirmed through data a few things we already know:

- China being the most populated
- India being most to add population by number

Also, we learnt a few interesting facts like below:

- Eastern European countries of Bulgaria, Serbia, Latvia having net highest death to birth ratio
- South Sudan has the highest population growth rate